<a href="https://colab.research.google.com/github/jdiazram/lecturasIIC3633-2021/blob/main/practicos/pyRecLab_SlopeOne_JDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://youtu.be/A2euuevpYis" target="_parent"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/YouTube_full-color_icon_%282017%29.svg/71px-YouTube_full-color_icon_%282017%29.svg.png" alt="Open In Colab"/></a>


# **Práctico Sistemas Recomendadores: pyreclab - Slope One**

En este práctico seguiremos utilizando [pyreclab](https://github.com/gasevi/pyreclab), con el cual estamos aprendiendo distintas técnicas de recomendación. Seguiremos usando la misma base de datos de los prácticos anteriores, para que puedan comparar los métodos y sus implementaciones. Este práctico está acompañado de un [video comentando la actividad](https://youtu.be/A2euuevpYis).

En esta oportunidad exploraremos el recomendador de Pendiente Uno o **Slope One** [1].

**Adaptado y preparado por:** Francisca Cattan 📩 fpcattan@uc.cl

Referencias 📖
------
[1] *Lemire, D., & Maclachlan, A. (2005, April). Slope One Predictors for Online Rating-Based Collaborative Filtering. In SDM (Vol. 5, pp. 1-5).*


**Nombre**:  Jorge Díaz Ramírez

## Actividad 1 👓

Antes de empezar con el práctico, responde la siguiente pregunta con lo visto en clases.

**Pregunta:** Explique cómo funciona Slope One (como modelo teórico, no piense en la implementación). En particular explique:

- Repasemos: ¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?
- Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?
- ¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)
- ¿Qué pasaría si se agrega un nuevo rating a la base de datos?
- Opcional: ¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?

💡 *Hint: La bibliografía todo lo puede.*

**Respuesta:**

- El algoritmo Slope One calcula la diferencia media entre los ratings de 2 items para los usuarios que calificaron ambos items. Esto con el fin de realizar predicciones de items a otros usuarios y es el mas simple de los algoritmos de filtrado colaborativo. Respondiendo a la pregunta **¿Por qué este recomendador es un algoritmo de Filtrado Colaborativo?** Slope One se basa en los ratings de items para recomendar items a usuarios.
- **Este Filtrado Colaborativo, ¿está basado en el usuario o en los items? ¿Por qué?** Slope one esta basado en items, porque calcula la diferencia media de ratings entre items.
- **¿Qué datos recibe Slope One y qué hace con ellos? (qué tipo de columnas y qué calculo)** Los datos que recibe son: el usuario, item y rating. La columna sería el item, las filas los usuarios y la intersección sería el rating que le dió el usuario al item. El cálculo es la **Resta** del rating del item J menos el rating de item I y esa diferencia se le agrega a otro usuario para predecir cual sería el rating del item J (faltante) para ese otro usuario.
-**¿Qué pasaría si se agrega un nuevo rating a la base de datos?** Como se utilizan diferencias promedio entre rating, al agregar un nuevo rating variaría la diferencia, modificando la predicción del rating.
- Opcional: **¿Cómo crees que le iría al recomendador con un usuario que acaba de entrar al sistema y ha asignado muy pocos ratings?** Creo que bien, ya que todo dependería de los otros usuarios (antiguos) hayan valorizado o agregado un rating a los items que el nuevo usuario no haya valorizado, con esto se podría tener una predicción de los items faltantes para el nuevo usuario.






# **Configuración Inicial**

## Paso 1:
Descargue directamente a Colab los archivos del dataset ejecutando las siguientes 3 celdas:


In [1]:
!curl -L -o "u1.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    330      0 --:--:--  0:00:01 --:--:--   331
100 1546k  100 1546k    0     0   883k      0  0:00:01  0:00:01 --:--:--  883k


In [2]:
!curl -L -o "u1.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    477      0 --:--:-- --:--:-- --:--:--   477
100  385k  100  385k    0     0   288k      0  0:00:01  0:00:01 --:--:--  288k


In [3]:
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    554      0 --:--:-- --:--:-- --:--:--   553
100  230k  100  230k    0     0   191k      0  0:00:01  0:00:01 --:--:--  191k


Los archivos **u1.base** y **u1.test** tienen tuplas {usuario, item, rating, timestamp}, que es la información de preferencias de usuarios sobre películas en una muestra del dataset [movielens](https://grouplens.org/datasets/movielens/).

## Paso 2:

Instalamos pyreclab utilizando pip.

In [4]:
!pip install pyreclab --upgrade

     |████████████████████████████████| 234 kB 4.2 MB/s 


## Paso 3:

Hacemos los imports necesarios para este práctico.

In [5]:
import pyreclab
import numpy as np
import pandas as pd

# **El dataset**

💡 *En prácticos anteriores, vimos como analizar este dataset. Puedes revisarlos en caso de dudas.*

## Paso 4:

Ya que queremos crear una lista de recomendación de items para un usuario en especifico, necesitamos obtener información adicional de cada película tal como título, fecha de lanzamiento, género, etc. Cargaremos el archivo de items descargado "u.item" para poder mapear cada identificador de ítem al conjunto de datos que lo describe.

In [6]:
# Definimos el orden de las columnas
info_cols = [ 'movieid', 'title', 'release_date', 'video_release_date', 'IMDb_URL', \
              'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', \
              'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', \
              'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western' ]

# Asignamos a una variable la estructura de datos de los items
info_file = pd.read_csv('u.item', sep='|', index_col = 0, names = info_cols, header=None, encoding='latin-1')

# **Slope One**

## Paso 5:

Seguiremos un camino muy similar a los ejercicios de User KNN e Item KNN. Crearemos una instancia del algoritmo de recomendación y luego pasaremos a la fase de entrenamiento.

In [7]:
# Declaramos la instancia SlopeOne
mySlopeOne = pyreclab.SlopeOne(dataset='u1.base', dlmchar=b'\t', header=False, usercol=0, itemcol=1, ratingcol=2)

In [8]:
# Y enntrenamos
mySlopeOne.train()

## Actividad 2 👓

**Pregunta:** Explique qué hace el método `train()` en este caso, dado el modelo teórico. ¿Calcula información?, ¿no hace nada?, ¿ordena los datos? 

**Respuesta:**
Calcula todas las diferencias promedio de los ratings de item por usuario de todo el dataset u1.base

## Paso 6:

Llego la hora de predecir el rating.

In [9]:
# Esta es la predicción de rating que el usuario ID:457 otorgaría al ítem ID:37
# De esta forma podemos comparar el resultado con los prácticos anteriores
mySlopeOne.predict("457", "37")

3.2408759593963623

In [10]:
# También podemos guardar la predicción en una variable
prediction = mySlopeOne.predict("457", "37")

In [11]:
# Podemos comprobar las peliculas rankeadas por el usuario ID:457
# Que ciertamente ha participado activamente (¡156 items!)
train_file = pd.read_csv('u1.base', sep='\t', names = ['userid', 'itemid', 'rating', 'timestamp'], header=None)
train_file[train_file['userid'] == 457]

,userid,itemid,rating,timestamp
37269,457,1,4,882393244
37270,457,7,4,882393278
37271,457,9,5,882393485
37272,457,11,4,882397020
37273,457,13,3,882393883
...,...,...,...,...
37420,457,1047,2,882395964
37421,457,1119,4,882398308
37422,457,1168,5,882548761
37423,457,1210,4,882549905


In [12]:
# Y también cuáles usuarios han rankeado la pelicula ID:37
train_file[train_file['itemid'] == 37]

,userid,itemid,rating,timestamp
1302,13,37,1,882397011
14851,201,37,2,884114635
19670,268,37,3,876514002
29489,363,37,2,891498510
31084,385,37,4,880013483
32996,405,37,1,885548384
62777,773,37,3,888540352


## Actividad 3 👓

Haremos un pequeño experimento para entender mejor como funciona Slope One. Gracias al ejercicio anterior, sabemos que el usuario 457 ya ha asignado el mejor rating (5 ⭐) a las dos peliculas ID:9 e ID:1168. Comparemos.

**Pregunta:** ¿Cómo se explican estos resultados?  

**Respuesta:** abajo

In [13]:
prediction_id9 = mySlopeOne.predict("457", "9")
prediction_id1168 = mySlopeOne.predict("457", "1168")

print('Prediction for ID:9 :', prediction_id9)
print('Prediction for ID:1168 :', prediction_id1168)

Prediction for ID:9 : 4.530702114105225
Prediction for ID:1168 : 4.166153907775879


In [14]:
#para ver los otros rating de item 9
train_file[train_file['itemid'] == 9]

,userid,itemid,rating,timestamp
449,6,9,4,883599205
604,7,9,5,891351432
976,10,9,4,877889005
1116,11,9,5,891902970
1291,13,9,3,882140205
...,...,...,...,...
79427,937,9,5,876769373
79469,938,9,3,891356413
79575,939,9,5,880260745
79627,940,9,3,885921687


In [16]:
#para ver los otros rating de item 1168
train_file[train_file['itemid'] == 1168]

,userid,itemid,rating,timestamp
6664,85,1168,3,882995908
9646,122,1168,4,879270902
20141,269,1168,2,891448386
21600,280,1168,5,891702544
29685,363,1168,2,891496587
30647,378,1168,3,880333383
30899,380,1168,3,885479833
31698,389,1168,3,880088717
32183,393,1168,3,889729346
34386,416,1168,4,886318953


**Respuesta:** Si bien el usuario 457 le dió 5 estrellas a los item 9 y 1168, en la predicción aparecen para item 9: 4.53 y para item 1168: 4.16, esto es debido al algoritmo de slope one, ya que al realizar las diferencias media, depende de esos rating para calcular la prediccióón. En celda anterior se observan los otros rating para el item 9 e item 1168, observando mayor cantidad de rating menos de 5, para ambos casos.

## Paso 7:

Generaremos ahora una lista ordenada de las top-N recomendaciones, dado un usuario.



In [17]:
# Mediante el método recommend() genereremos una lista top-5 recomendaciones para el usuario ID:457
reclist_slopeone = mySlopeOne.recommend("457", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone)

Lista de items según ID: ['1592', '1589', '1656', '1431', '1653']


In [18]:
# Lo convertimos a numpy array
recmovies_slopeone = np.array(reclist_slopeone).astype(int)

# Utilizamos la estructura de datos de los items para encontrar los títulos recomendados
print('Lista de items por nombre:')
info_file.loc[recmovies_slopeone]['title']

Lista de items por nombre:


movieid
1592                               Magic Hour, The (1998)
1589                                   Schizopolis (1996)
1656                                   Little City (1998)
1431                                  Legal Deceit (1997)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## Actividad 4 👩🏻‍💻

Genera una nueva recomendacion, modificando los hiperparametros de usuario y topN a tu elección.

**Pregunta:** ¿Ves una diferencia en la recomendación entre el nuevo usuario y el usuario ID:457?

**Respuesta:** abajo

In [21]:
# Escribe el nuevo codigo aqui
#nuevo usuario
reclist_slopeone_new = mySlopeOne.recommend("85", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone_new)

Lista de items según ID: ['1536', '1500', '47', '46', '44']


In [22]:
train_file[train_file['userid'] == 85]

,userid,itemid,rating,timestamp
6457,85,8,4,879454952
6458,85,9,4,879456308
6459,85,13,3,879452866
6460,85,23,4,879454272
6461,85,25,2,879452769
...,...,...,...,...
6663,85,1167,3,879829209
6664,85,1168,3,882995908
6665,85,1169,4,879454952
6666,85,1171,3,879452638


**Respuesta**: si se observan diferencias, ya que depende de que valoraciones (rating) que se le hayan dado a los items, pero por otros usuarios, que hayan consumido esos items, similares a los rating dados por este usuario.

## Actividad 5 👩🏻‍💻

Dado el usuario ID:44, cree dos listas de películas recomendadas; la primera utilizando el algoritmo Most Popular y la segunda utilizando el algoritmo Slope One.

**Pregunta:** Realice un analisis apreciativo de las similitudes y diferencias entre ambas recomendaciones.

**Respuesta:**Abajo

Most Popular

In [26]:
# Escribe el nuevo codigo aqui
# Definicion de objeto "most popular"
most_popular = pyreclab.MostPopular(dataset='u1.base',
                   dlmchar=b'\t',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

# en teoria no es "entrenar" , objeto "most_popular" solo registra los items más populares en el set de entrenamiento  
most_popular.train()

In [27]:
# Calcular las recomendaciones para un usuario en particular (id =2)
user_id = 44
top_n = 5

ranking = [int(r) for r in most_popular.recommend(str(user_id), top_n, includeRated=False)]
print('Recommendation for user {}: {}'.format(user_id, ranking))

Recommendation for user 44: [50, 100, 181, 286, 1]


Slope One

In [28]:
# Escribe el nuevo codigo aqui
#nuevo usuario
reclist_slopeone_new1 = mySlopeOne.recommend("44", 5)

# Y visualizaremos el resultado
print('Lista de items según ID:', reclist_slopeone_new1)

Lista de items según ID: ['1656', '1064', '1643', '1642', '1625']


**Respuesta:** El algoritmo de **Most Popular** se basa en los items más populares por lo que con **Slope One**, que se basa en diferencias medias, se diferencia en la forma de realizar la predicción o recomendación para el usuario, y esto se observa en los resultados anteriores, dando recomendaciones distintas para el mismo usuario. La similitud es que se basan en los items para dar alguna recomendación.